In [1]:
!pip install gensim==3.8.3

In [2]:
import gensim
print(gensim.__version__)

3.8.3


In [58]:
import re
import os
import random
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
import matplotlib.pyplot as plt
import seaborn as sns
import gensim.downloader as api

from sklearn.model_selection import train_test_split
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from tqdm.notebook import tqdm

In [59]:
ChatbotData = pd.read_csv('~/aiffel/transformer_chatbot/data/ChatbotData.csv')
ChatbotData

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [60]:
ChatbotData.shape

(11823, 3)

In [61]:
ChatbotData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [62]:
train_data, test_data = train_test_split(ChatbotData, test_size=0.1, random_state=42)

In [64]:
# 각 데이터에서 질문(Q)과 답변(A) 분리
train_que = train_data['Q'].tolist()
train_ans = train_data['A'].tolist()

test_que = test_data['Q'].tolist()
test_ans = test_data['A'].tolist()

print(f"훈련 데이터 수: {len(train_que)}")
print(f"테스트 데이터 수: {len(test_que)}")

훈련 데이터 수: 10640
테스트 데이터 수: 1183


In [65]:
# 결과 확인
print("Sample Questions:", questions[:3])
print("Sample Answers:", answers[:3])

Sample Questions: ['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다']
Sample Answers: ['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.']


In [66]:
def preprocess_sentence(sentence):
    # 소문자로 변환
    sentence = sentence.lower()

    # 영문자, 한글, 숫자, 공백, 주요 특수문자(.,!?~)만 남기고 제거
    sentence = re.sub(r'[^a-zA-Z0-9가-힣\s.,!?~]', '', sentence)
    
    # 문자열 양쪽 공백 제거
    sentence = sentence.strip()

    return sentence

In [67]:
# TEST
test_sentence = "Hello! 안녕하세요? 😊 연습입니다!!"
print(preprocess_sentence(test_sentence))

hello! 안녕하세요?  연습입니다!!


In [68]:
# Mecab 객체 생성
mecab = Mecab()

def build_corpus(source_data, target_data, tokenize_fn, max_token_length=50):
    """
    데이터를 정제하고 토큰화하여 말뭉치를 생성하는 함수.
    
    Args:
        source_data (list of str): 소스 문장 리스트 (질문)
        target_data (list of str): 타겟 문장 리스트 (답변)
        tokenize_fn (function): 사용할 토크나이즈 함수 (ex: mecab.morphs)
        max_token_length (int): 최대 토큰 길이 (기본값: 50)
    
    Returns:
        tuple: (소스 말뭉치, 타겟 말뭉치)
    """
    def preprocess_and_tokenize(data):
        processed = set()  # 중복 제거를 위한 집합
        tokenized_data = []  # 최종 토큰화된 문장 저장
        
        for sentence in data:
            # 1. 문장 정제
            cleaned_sentence = preprocess_sentence(sentence)

            # 2. 문장 토큰화
            tokens = tokenize_fn(cleaned_sentence)

            # 3. 최대 토큰 길이 초과 시 제외
            if len(tokens) <= max_token_length:
                # 4. 중복된 문장은 추가하지 않음
                joined_tokens = ' '.join(tokens)  # 토큰을 문자열로 변환해 중복 검사
                if joined_tokens not in processed:
                    processed.add(joined_tokens)
                    tokenized_data.append(tokens)

        return tokenized_data

    # 소스와 타겟 데이터 각각에 대해 처리
    que_corpus = preprocess_and_tokenize(source_data)
    ans_corpus = preprocess_and_tokenize(target_data)

    return que_corpus, ans_corpus

In [69]:
# build_corpus 함수를 사용하여 말뭉치 생성
que_corpus, ans_corpus = build_corpus(questions, answers, mecab.morphs)

# 결과 확인
print("Questions Corpus Sample:", que_corpus[:3])
print("Answers Corpus Sample:", ans_corpus[:3])

Questions Corpus Sample: [['12', '시', '땡', '!'], ['1', '지망', '학교', '떨어졌', '어'], ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다']]
Answers Corpus Sample: [['하루', '가', '또', '가', '네요', '.'], ['위로', '해', '드립니다', '.'], ['여행', '은', '언제나', '좋', '죠', '.']]


In [49]:
# Word2Vec 모델 로드
word2vec = KeyedVectors.load_word2vec_format('~/aiffel/transformer_chatbot/data/ko.bin', binary=True)

def lexical_sub(sentence, wv):
    """
    문장에서 임의의 단어를 유사한 단어로 교체하는 함수.
    Args:
        sentence (list of str): 토큰화된 입력 문장
        wv (gensim.models.KeyedVectors): Word2Vec 임베딩 모델
    Returns:
        list of str: 유사 단어가 교체된 새로운 문장 (혹은 원본 문장)
    """
    # Word2Vec에 있는 단어만 선택
    valid_tokens = [tok for tok in sentence if tok in wv]

    # 유효한 단어가 없으면 원본 문장 반환
    if not valid_tokens:
        return sentence

    # 랜덤으로 교체할 단어 선택
    selected_tok = random.choice(valid_tokens)

    # 유사한 단어로 교체한 새로운 문장 생성
    new_sentence = [
        wv.most_similar(tok, topn=1)[0][0] if tok == selected_tok else tok for tok in sentence
    ]

    return new_sentence

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

### utf-8의 문제
- ko.bin을 불러올 때마다 utf-8 인코딩 에러가 계속 발생
- 여러 방법으로 고쳐볼려 했지만 실패
- 버전을 다운그레이드하니 해결

In [70]:
model_path = '~/aiffel/transformer_chatbot/data/ko.bin'

word2vec = Word2Vec.load(model_path).wv 

# Lexical Substitution 함수 정의
def lexical_sub(sentence, wv):
    """
    문장에서 임의의 단어를 유사한 단어로 교체하는 함수.
    Args:
        sentence (list of str): 토큰화된 입력 문장
        wv (gensim.models.KeyedVectors): Word2Vec 임베딩 모델
    Returns:
        list of str: 유사 단어가 교체된 새로운 문장 (혹은 원본 문장)
    """
    # Word2Vec에 있는 단어만 선택
    valid_tokens = [tok for tok in sentence if tok in wv]

    # 유효한 단어가 없으면 원본 문장 반환
    if not valid_tokens:
        return sentence

    # 랜덤으로 교체할 단어 선택
    selected_tok = random.choice(valid_tokens)

    # 유사한 단어로 교체한 새로운 문장 생성
    new_sentence = [
        wv.most_similar(tok, topn=1)[0][0] if tok == selected_tok else tok for tok in sentence
    ]

    return new_sentence

In [71]:
# 새로운 말뭉치 초기화
new_que_corpus = []
new_ans_corpus = []

# 1. 질문(Questions) 말뭉치에 대해 Augmentation 수행
for tokens in tqdm(que_corpus, desc="Augmenting Questions"):
    new_tokens = lexical_sub(tokens, word2vec)
    new_que_corpus.extend([new_tokens, tokens])  # 증강된 문장과 원본 문장 추가

# 2. 답변(Answers) 말뭉치에 대해 Augmentation 수행
for tokens in tqdm(ans_corpus, desc="Augmenting Answers"):
    new_tokens = lexical_sub(tokens, word2vec)
    new_ans_corpus.extend([new_tokens, tokens])  # 증강된 문장과 원본 문장 추가

# 병렬 데이터 생성: (Aug Q, Original A), (Original Q, Aug A), (Original Q, Original A)
augmented_data = []

# 최대 길이로 맞추기 위해 한쪽 데이터가 부족할 경우 반복 사용
max_pairs = max(len(que_corpus), len(ans_corpus))

# 3배 데이터 생성
for i in range(max_pairs):
    # i가 범위를 초과할 경우 무작위 데이터 사용
    q_idx = i % len(que_corpus)
    a_idx = i % len(ans_corpus)

    # (Augmented Q, Original A)
    augmented_data.append((new_que_corpus[2 * q_idx], ans_corpus[a_idx]))

    # (Original Q, Augmented A)
    augmented_data.append((que_corpus[q_idx], new_ans_corpus[2 * a_idx]))

    # (Original Q, Original A)
    augmented_data.append((que_corpus[q_idx], ans_corpus[a_idx]))

# 데이터 개수 확인 및 예시 출력
print(f"총 생성된 데이터 수: {len(augmented_data)}")

print("첫 번째 데이터 쌍:")
print("Q:", ' '.join(augmented_data[0][0]))
print("A:", ' '.join(augmented_data[0][1]))

Augmenting Questions:   0%|          | 0/11643 [00:00<?, ?it/s]

Augmenting Answers:   0%|          | 0/7728 [00:00<?, ?it/s]

총 생성된 데이터 수: 34929
첫 번째 데이터 쌍:
Q: 12 시 끗 !
A: 하루 가 또 가 네요 .


In [72]:
# <start>와 <end> 토큰을 ans_corpus에 추가
ans_corpus = [["<start>"] + tokens + ["<end>"] for tokens in ans_corpus]

# 예시 출력
print("첫 번째 타겟 문장:", ans_corpus[0])

첫 번째 타겟 문장: ['<start>', '하루', '가', '또', '가', '네요', '.', '<end>']


In [73]:
# 모든 질문과 답변 데이터를 결합하여 단어 사전 구축
all_sentences = que_corpus + ans_corpus

# 단어 빈도를 기반으로 단어 사전 생성
vocab = set([token for sentence in all_sentences for token in sentence])

# 단어 -> 인덱스 매핑 및 반대 매핑 생성
word2idx = {word: idx + 1 for idx, word in enumerate(vocab)}  # 인덱스는 1부터 시작
word2idx["<pad>"] = 0  # 패딩 토큰 추가
idx2word = {idx: word for word, idx in word2idx.items()}

# 단어 사전 크기 출력
print(f"단어 사전 크기: {len(word2idx)}")

단어 사전 크기: 6834


In [74]:
def vectorize_corpus(corpus, word2idx, max_len=20):
    """
    말뭉치를 벡터화하는 함수.
    Args:
        corpus (list of list of str): 토큰화된 문장 리스트
        word2idx (dict): 단어 -> 인덱스 매핑 딕셔너리
        max_len (int): 최대 문장 길이 (기본값: 20)
    Returns:
        np.ndarray: 벡터화된 문장 배열 (패딩 포함)
    """
    vectorized_data = []
    for sentence in corpus:
        vector = [word2idx.get(token, word2idx["<pad>"]) for token in sentence]
        # max_len에 맞춰 패딩 추가 또는 자르기
        vector = vector[:max_len] + [word2idx["<pad>"]] * (max_len - len(vector))
        vectorized_data.append(vector)
    return np.array(vectorized_data)

# 질문과 답변 데이터 벡터화
enc_train = vectorize_corpus(que_corpus, word2idx)
dec_train = vectorize_corpus(ans_corpus, word2idx)

# 데이터 형태 확인
print(f"enc_train shape: {enc_train.shape}")
print(f"dec_train shape: {dec_train.shape}")

enc_train shape: (11643, 20)
dec_train shape: (7728, 20)


In [75]:
print("첫 번째 인코더 입력:", enc_train[0])
print("첫 번째 디코더 입력:", dec_train[0])

# 인덱스를 단어로 변환해보기
print("첫 번째 질문 복원:", [idx2word[idx] for idx in enc_train[0]])
print("첫 번째 답변 복원:", [idx2word[idx] for idx in dec_train[0]])

첫 번째 인코더 입력: [1198 3349 2740 5278    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
첫 번째 디코더 입력: [5963 3498 3659 4986 3659 2812  207 2638    0    0    0    0    0    0
    0    0    0    0    0    0]
첫 번째 질문 복원: ['12', '시', '땡', '!', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
첫 번째 답변 복원: ['<start>', '하루', '가', '또', '가', '네요', '.', '<end>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [103]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [104]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [105]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [106]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [107]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [108]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [109]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [110]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [111]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [112]:
VOCAB_SIZE = len(word2idx)

transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
		
d_model = 512

In [113]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [114]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [115]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [116]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [118]:
EPOCHS = 3

for epoch in range(EPOCHS):
    total_loss = 0
    
    dataset_count = tf.data.experimental.cardinality(augmented_data).numpy()
    tqdm_bar = tqdm(enumerate(augmented_data), total=dataset_count, desc=f'Epoch {epoch+1}/{EPOCHS}')

    for batch, (src, tgt) in tqdm_bar:
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(src, tgt, transformer, optimizer)  

        total_loss += batch_loss
        
        tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
        tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1))) 

AttributeError: 'list' object has no attribute '_variant_tensor'